# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
#импортируем необходимые библиотеки
import pandas as pd
from pymystem3 import Mystem
m = Mystem()

In [2]:
#прочитаем файл с данными
data = pd.read_csv('/datasets/data.csv')
#просмотр общей информации о данных
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
#поиск пропущенных значений
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [4]:
#ознакомимся с таблицей
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывод**

Переименование столбцов не требуется.<br>
В столбцах `days_employed` и `total_income` имеются явные пропуски (по 2174 строк). Количество строк с пропусками составляет примерно 10%, процент слишком большой - строки удалить нельзя, необходимо заполнить пропуски.<br>
Во многих столбцах тип данных `object` - необходим анализ данных этих столбцов и, при необходимости, перевод в "целочисленный" вид.<br> В связи с повторяющимися целями кредита в столбце `purpose` целесообразно провести лемматизацию и категоризацию. 

## Шаг 2. Предобработка данных

### Обработка пропусков

In [5]:
#посмотрим общие данные числовых столбцов таблицы
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


1. Как видно, в столбце `days_employed` много отрицательных значений и есть очень большие значения, не соответствующие действительности - это явные ошибки. Отрицательные значения можно взять по модулю, а слишком большие значения  и пропуски заменить средним. Но в связи с тем, что данные этого столбца не нужны для выполнения заданий данного проекта, удалим столбец `days_employed`. 
2. В столбце `total_income` содержатся сведения о доходах, в этом случае пропуски лучше заполнять медианным значением, ведь в данных могут быть выдающиеся значения, которые сместят среднее. Причем заполним пропуски медианым значением для каждой группы столбца `income_type`.
3. В столбце `dob_years` имеются нулевые значения, но они составляют менее 0,5%. Видимо нулями заполнены пропуски. Данные этого столбца не нужны для выполнения заданий нашего проекта, ошибки можно не исправлять.

In [6]:
# подсчет нулевых значений в столбце 'dob_years'
data[data['dob_years'] == 0]['dob_years'].count()

101

In [7]:
#подсчет уникальных значений столбца "gender"
data['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [8]:
#замена значения "XNA" на "F" столбца "gender", увеличение значения 'F' на единицу ничего не изменит.
data['gender'] = data['gender'].replace('XNA', 'F')

In [9]:
#удаление столбца "days_employed"
data.drop(columns = 'days_employed', inplace = True)

In [10]:
#подсчет уникальных значений столбца "income_type"
data['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64

Доля людей со статусами 'предприниматель', 'безработный', 'студент' и 'в декрете' очень мала. Если бы данные этого столбца были нам нужны в дальнейшем, то можно было бы присоединить эти строки к более многочисленным статусам или вовсе удалить. Оставим все как есть.

In [11]:
#заполним пропуски медианым значением для каждой группы столбца "income_type"
data['total_income'] = data['total_income'].fillna(data.groupby('income_type')['total_income'].transform('median'))

In [12]:
#посмотрим информацию о таблице еще раз
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 11 columns):
children            21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(1), int64(5), object(5)
memory usage: 1.8+ MB


In [13]:
#посмотрим данные после заполнения пропусков в столбце 'total_income'
data.describe()

,children,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,2.152500e+04
mean,0.538908,43.293380,0.817236,0.972544,0.080883,1.652253e+05
std,1.381587,12.574584,0.548138,1.420324,0.272661,9.804367e+04
min,-1.000000,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,33.000000,1.000000,0.000000,0.000000,1.077982e+05
50%,0.000000,42.000000,1.000000,0.000000,0.000000,1.425944e+05
75%,1.000000,53.000000,1.000000,1.000000,0.000000,1.955499e+05
max,20.000000,75.000000,4.000000,4.000000,1.000000,2.265604e+06


**Вывод**

Как видно, явные пропуски в таблице устранены. Данные столбца `total_income` нужно перевести из вещественного типа в целый.

### Замена типа данных

In [14]:
#перевод в целочисленное значение столбца 'total_income' после преобразования данных
data['total_income'] = data['total_income'].astype('int')

#проверка данных
data['total_income'].dtype

dtype('int64')

In [15]:
#вывод уникальных значений столбца 'children' и их количества
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

В данных видны явные ошибки: 
- количество детей не может быть отрицательным; 
- слишком большое количество человек (76) имеет 20 детей. На сколько мне известно, в РФ есть всего одна семья с таким количеством родных детей.<br>Заменим значение "-1" на "1", "20" на "2".</br>

In [16]:
#Заменим значение '-1' на '1', '20' на '2'.
data.loc[data['children'] == -1,'children'] = '1'
data.loc[data['children'] == 20,'children'] = '2'

#перевод в целочисленное значение после преобразования данных
data['children'] = pd.to_numeric(data['children'])

In [17]:
#вывод уникальных значений столбца 'children' и их количества после преобразования данных
data['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

**Вывод**

1. Преобразовали тип данных столбца `total_income` в целочисленное значение.
2. Перевод в целочисленное значение данных столбца `children` после исправления ошибок.

### Обработка дубликатов

In [18]:
#Посмотрим уникальные значения столбца 'education'
unique_education = data['education'].unique()
unique_education

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [19]:
#В столбце очень много повторяющихся значений, написанных разным регистром. Приведем их к одному виду.
data['education'] = data['education'].str.lower()
#подсчитаем уникальные значения столбца 'education' и их количество
data['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [20]:
# Посмотрим уникальные значения столбца 'education_id' и их количество
data['education_id'].value_counts()

1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64

Число строк(уникальных значений) совпадает, совпадает и количество в строках.<br> Значит:<br> 
0  - высшее;<br> 
1  - среднее;<br> 
2  - неоконченное высшее;<br>
3  - начальное;<br>
4  - ученая степень. <br>

In [21]:
#подсчитаем уникальные значения столбца 'family_status' и их количество
data['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [22]:
#подсчитаем уникальные значения столбца 'family_status_id' и их количество
data['family_status_id'].value_counts()

0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64

число строк(уникальных значений)совпадает, совпадает и количество в строках.<br> Значит:<br> 0 - женат / замужем;<br> 1 - гражданский брак;<br> 2 - вдовец / вдова;<br> 3 - в разводе;<br> 4 - Не женат / не замужем.<br> 

In [23]:
#найдем явные дубликаты и посчитаем их количество
data.duplicated().sum()

71

In [24]:
#удаление дубликатов со сбросом индекса строк
data = data.drop_duplicates().reset_index(drop = True)

#посмотрим информацию после удаления дубликатов
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 11 columns):
children            21454 non-null int64
dob_years           21454 non-null int64
education           21454 non-null object
education_id        21454 non-null int64
family_status       21454 non-null object
family_status_id    21454 non-null int64
gender              21454 non-null object
income_type         21454 non-null object
debt                21454 non-null int64
total_income        21454 non-null int64
purpose             21454 non-null object
dtypes: int64(6), object(5)
memory usage: 1.8+ MB


**Вывод**

Проведена обработка дубликатов:<br> 1. данные столбца `education` приведены к нижнему регистру, что позволило сократить количество уникальных значений с 15 до 5.<br> 2. При анализе данных столбцов `education` и `education_id` , `family_status` и `family_status_id` обнаружено, что они дублируют друг друга и при необходимости один из пары можно удалить(но для решения нашей задачи это не требуется).<br> 3. Удалены явные дубликаты строк(71 строка), что позволило уменьшить размер таблицы и объем используемой памяти. 

### Лемматизация

In [25]:
#Посмотрим уникальные значения столбца 'purpose'
unique_purpose = data['purpose'].unique()
unique_purpose

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

В столбце много однотипных целей, записанных по-разному. Проведем лемматизацию - напишем функцию, которая лемматизирует данные столбца `purpose` и разбивает цели кредита на категории (основные цели).

In [26]:
#напишем функцию для лемматизации цели получения кредита и разбивки цели на категории
def purpose_cat(purpose):
    lemmas_row = m.lemmatize(purpose)
    for i in lemmas_row:
        if 'авто' in i:
            return 'автомобиль'
        if 'недвижимость' in i:
            return  'недвижимость'
        if 'жилье' in i:
            return  'жилье'
        if 'свадьба' in i:
            return  'свадьба'
        if 'образование' in i:
            return 'образование'
    return 'Такая категория отсутствует' #проверка на ошибки

#добавим столбец 'purpose_cat' в таблицу  
data['purpose_cat']= data['purpose'].apply(purpose_cat)      

In [27]:
#проверим работу функции
print(purpose_cat('жилье'))
purpose_cat('лечение')

жилье


'Такая категория отсутствует'

In [28]:
#проверка. Выведем первые 11 строк, чтобы встречались все категории
data['purpose_cat'].head(11)

0            жилье
1       автомобиль
2            жилье
3      образование
4          свадьба
5            жилье
6            жилье
7      образование
8          свадьба
9            жилье
10    недвижимость
Name: purpose_cat, dtype: object

In [29]:
#подсчитаем на какие цели сколько раз брали кредит
data['purpose_cat'].value_counts()

недвижимость    6351
жилье           4460
автомобиль      4306
образование     4013
свадьба         2324
Name: purpose_cat, dtype: int64

**Вывод**

Проведена лемматизация целей кредита и разбивка на категории.<br> Исходя из полученных данных можно сказать, что 50% кредитов берут на жилье и недвижимость, по 20 % на автомобиль и образование и 10 % на свадьбу.

### Категоризация данных

In [30]:
#посмотрим общие данные числовых столбцов таблицы после выполнения предыдущих пунктов
data.describe()

,children,dob_years,education_id,family_status_id,debt,total_income
count,21454.000000,21454.000000,21454.000000,21454.000000,21454.000000,2.145400e+04
mean,0.480563,43.271231,0.817097,0.973898,0.081150,1.653196e+05
std,0.756069,12.570822,0.548674,1.421567,0.273072,9.818730e+04
min,0.000000,0.000000,0.000000,0.000000,0.000000,2.066700e+04
25%,0.000000,33.000000,1.000000,0.000000,0.000000,1.076230e+05
50%,0.000000,42.000000,1.000000,0.000000,0.000000,1.425940e+05
75%,1.000000,53.000000,1.000000,1.000000,0.000000,1.958202e+05
max,5.000000,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [31]:
#напишем функцию для разбивки дохода по категориям. В качестве значений будем использовать значения квартилей 
#для столбца 'total_income'
def salary(total_income):
    if (total_income > 20667) & (total_income <=107623):
        return 'доход меньше 107623 руб.'
    if (total_income > 107623) & (total_income <=142594):
        return '107623 руб. < доход <= 142594 руб.'
    if (total_income > 142594) & (total_income <=195820):
        return '142594 руб. < доход <= 195820 руб.'
    return 'доход больше 195820 руб.'

In [32]:
#проверка работы функции
print(salary(55000))
print(salary(108000))
print(salary(194000))
salary(240000)

доход меньше 107623 руб.
107623 руб. < доход <= 142594 руб.
142594 руб. < доход <= 195820 руб.


'доход больше 195820 руб.'

In [33]:
# создадим столбец с разбивкой дохода по категориям
data['salary'] = data['total_income'].apply(salary)

In [34]:
#проверим, что все строки столбца 'salary' разбились на категории
data['salary'].value_counts()

107623 руб. < доход <= 142594 руб.    5479
доход больше 195820 руб.              5365
доход меньше 107623 руб.              5363
142594 руб. < доход <= 195820 руб.    5247
Name: salary, dtype: int64

**Вывод**

С помощью категоризации разбили доход на группы для ответа на вопросы проекта.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [35]:
#создадим таблицу для определения зависимости между наличием детей и возвратом кредита в срок
data_grouped = data.groupby('children').agg({'debt':['count', 'sum']})

#расчет конверсии
data_grouped ['conversion'] = data_grouped['debt']['sum']/data_grouped['debt']['count']

#сортировка по конверсии
data_grouped.sort_values(by = 'conversion', ascending = False)

debt       conversion
          count   sum           
children                        
4            41     4   0.097561
2          2128   202   0.094925
1          4855   445   0.091658
3           330    27   0.081818
0         14091  1063   0.075438
5             9     0   0.000000

**Вывод**

Зависимости от наличия детей и возврата кредита в срок нет. Но из полученных данных можно сказать, что семьи с 5 детьми всегда рассчитываются с кредитом в срок (т.к. конверсия = 0). Процент просрочки кредита для семьи с 4-мя детьми составляет -  9,8%, с 2-мя детьми - 9,5%, с одним ребенком - 9,1%, с 3-мя детьми - 8,2 %, без детей - 7,5 %. Семьи с 5-ю детями и без детей расплачиваются с кредитом лучше, чем семьи с 1-м...4-мя детями. Семьи с 4-мя детями расплачиваются хуже всех.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [36]:
#создадим таблицу для определения зависимости между статусом семейного положения и возвратом кредита в срок
data_grouped_f = data.groupby('family_status').agg({'debt':['count', 'sum']})

#расчет конверсии
data_grouped_f['conversion'] = data_grouped_f['debt']['sum']/data_grouped_f['debt']['count']

#сортировка по конверсии
data_grouped_f.sort_values(by = 'conversion', ascending = False)

debt      conversion
                       count  sum           
family_status                               
Не женат / не замужем   2810  274   0.097509
гражданский брак        4151  388   0.093471
женат / замужем        12339  931   0.075452
в разводе               1195   85   0.071130
вдовец / вдова           959   63   0.065693

**Вывод**

Зависимости от семейного положения и возвратом кредита не наблюдается, т.к. во всех статусах семейного положения имеются просроченные платежи и это составляет 6,5-9.7%. Люди с семейным статусом "вдова/вдовец" имеют наименьший процент просрочки кредита, а люди находящиеся в "гражданском браке" и "холостые" чаще всего просрачивают кредит.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [37]:
#создадим таблицу для определения зависимости между уровнем дохода и возвратом кредита в срок
data_grouped = data.groupby('salary').agg({'debt':['count', 'sum']})

#расчет конверсии
data_grouped['conversion'] = data_grouped['debt']['sum']/data_grouped['debt']['count']

#сортировка по конверсии
data_grouped.sort_values(by = 'conversion', ascending = False)

debt      conversion
                                   count  sum           
salary                                                  
107623 руб. < доход <= 142594 руб.  5479  483   0.088155
142594 руб. < доход <= 195820 руб.  5247  448   0.085382
доход меньше 107623 руб.            5363  426   0.079433
доход больше 195820 руб.            5365  384   0.071575

**Вывод**

Люди имеющие максимальный и минимальный доход возвращают кредит в срок чаще, чем люди со средним доходом.

- Как разные цели кредита влияют на его возврат в срок?

In [38]:
#создадим таблицу для определения зависимости между целями кредита и возвратом кредита в срок
data_grouped = data.groupby('purpose_cat').agg({'debt':['count', 'sum']})

#расчет конверсии
data_grouped['conversion'] = data_grouped['debt']['sum']/data_grouped['debt']['count']

#сортировка по конверсии
data_grouped.sort_values(by = 'conversion', ascending = False)

debt      conversion
             count  sum           
purpose_cat                       
автомобиль    4306  403   0.093590
образование   4013  370   0.092200
свадьба       2324  186   0.080034
недвижимость  6351  474   0.074634
жилье         4460  308   0.069058

**Вывод**

Исходя из данных можно сказать, что чаще всего возвращают в срок кредит за жилье и недвижимость, реже всего за автомобиль.

## Шаг 4. Общий вывод

В данной работе выполнены следующие операции:<br>
найдены и заполнены пропуски;<br>
удален ненужный столбец `days_employed` с пропусками и некорретными данными;<br>
заменен вещественный тип данных на целочисленный;<br>
удалены дубликаты, переведены значения в строчный регистр;<br>
проведена лемматизация столбца `purpose` и категоризация столбцов `purpose` и `total_income`.<br>

Даны ответы на вопросы проекта:<br> 
`Есть ли зависимость между наличием детей и возвратом кредита в срок?` Семьи с 5-ми рассичтываются с кредитом в срок, а семьи с 4-мя детьми имеют самый большой процент просрочки кредита.<br>
`Есть ли зависимость между семейным положением и возвратом кредита в срок?` Люди со статусом "вдова/вдовец" имеют наименьший процент просрочки кредита, чем холостые.<br>
`Есть ли зависимость между уровнем дохода и возвратом кредита в срок?` Люди с максимальнным доходом имеют наименьший процент просрочки, чем люди со средним доходом.<br>
`Как разные цели кредита влияют на его возврат в срок?` Наиболее возвращаемым кредитом конечно же является кредит за жилье и недвижимость, а реже всего - за автомобиль.